In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import os
import glob

In [3]:
wcup = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Assorted-Projects\Datasets\WorldCUP\WorldCupMatches.csv')

In [4]:
wcup.head(3)

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA


In [8]:
oppo_list = glob.glob(r'C:\Users\USUARIO\Documents\GitHub\Assorted-Projects\Datasets\WorldCUP\archive\International_Football_History_Countries\*\Opponents_History.csv')

In [9]:
opponent_history = pd.DataFrame()

for i,path in enumerate(oppo_list):
    team = pd.read_csv(path)
    team['Main'] = oppo_list[i].split('\\')[10]
    opponent_history = pd.concat([opponent_history, team], axis=0)

In [10]:
opp_history = opponent_history[opponent_history.Squad != 'Squad']

In [12]:
opp_history.head()

,Squad,MP,W,D,L,GF,GA,GD,PPM,Head-to-Head,Main
0,cl Chile,19,11,7,1,30,13,+17,2.05,Head-to-Head,Argentina
1,uy Uruguay,17,11,3,3,27,15,+12,2.06,Head-to-Head,Argentina
2,br Brazil,17,5,4,8,15,26,-11,1.06,Head-to-Head,Argentina
3,bo Bolivia,16,11,3,2,41,16,+25,2.25,Head-to-Head,Argentina
4,co Colombia,16,8,6,2,20,10,+10,1.88,Head-to-Head,Argentina


In [15]:
bra_wc = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Assorted-Projects\Datasets\WorldCUP\archive\International_Football_History_Countries\Brazil\Worldcups_History.csv')
bra_wc.head()

,Season,Squad,Comp,LgRank,MP,W,D,L,GF,GA,GD,Pts,Top Team Scorer,Goalkeeper,Notes
0,2022,Brazil,1. FIFA World Cup,GR,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,2018,Brazil,1. FIFA World Cup,QF,5,3,1,1,8,3,5,10,"Philippe Coutinho, Neymar - 2",Alisson,NaN
2,2014,Brazil,1. FIFA World Cup,4th,7,3,2,2,11,14,-3,11,Neymar - 4,Júlio César,NaN
3,2010,Brazil,1. FIFA World Cup,QF,5,3,1,1,9,4,5,10,Luís Fabiano - 3,Júlio César,NaN
4,2006,Brazil,1. FIFA World Cup,QF,5,4,0,1,10,2,8,12,Ronaldo - 3,Dida,NaN


In [29]:
all_results = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Assorted-Projects\Datasets\WorldCUP\results.csv')
all_results

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
44055,2022-09-27,Norway,Serbia,0.0,2.0,UEFA Nations League,Oslo,Norway,False
44056,2022-09-27,Sweden,Slovenia,1.0,1.0,UEFA Nations League,Stockholm,Sweden,False
44057,2022-09-27,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,Pristina,Kosovo,False
44058,2022-09-27,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,Athens,Greece,False


In [30]:
all_results.dropna(inplace=True)

In [39]:
all_results['date'] = pd.to_datetime(all_results.date, infer_datetime_format=True)

In [47]:
ranking_fifa = pd.read_csv(r'C:\Users\USUARIO\Documents\GitHub\Assorted-Projects\Datasets\WorldCUP\fifa_ranking-2022-10-06.csv')

In [49]:
ranking_fifa['rank_date'] = pd.to_datetime(ranking_fifa.rank_date, infer_datetime_format=True)

In [52]:
ranking_fifa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63916 entries, 0 to 63915
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   rank             63916 non-null  int64         
 1   country_full     63916 non-null  object        
 2   country_abrv     63916 non-null  object        
 3   total_points     63916 non-null  float64       
 4   previous_points  63916 non-null  float64       
 5   rank_change      63916 non-null  int64         
 6   confederation    63916 non-null  object        
 7   rank_date        63916 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 3.9+ MB


In [55]:
ranking_fifa.rank_date[0].year

1992

In [57]:
ranking_fifa['Year'] = pd.DatetimeIndex(ranking_fifa.rank_date).year

In [60]:
rank_avg_year = ranking_fifa.groupby(['Year', 'country_full']).mean()

In [63]:
rank_avg_year.reset_index(inplace=True)

In [67]:
rank_avg_year[rank_avg_year.country_full == 'Ecuador']

,Year,country_full,rank,total_points,previous_points,rank_change
37,1992,Ecuador,65.000000,23.000000,0.000000,0.000000
186,1993,Ecuador,42.400000,37.000000,34.600000,-3.400000
343,1994,Ecuador,49.300000,33.500000,33.800000,0.700000
514,1995,Ecuador,60.500000,32.600000,32.100000,0.000000
692,1996,Ecuador,38.400000,43.800000,42.800000,-2.200000
879,1997,Ecuador,32.400000,50.600000,49.900000,-0.500000
1069,1998,Ecuador,44.700000,47.000000,48.300000,3.500000
1266,1999,Ecuador,57.333333,499.666667,463.333333,0.166667
1469,2000,Ecuador,64.916667,494.916667,489.666667,-0.916667
1671,2001,Ecuador,46.333333,575.416667,569.333333,-1.416667
